In [2]:
def get_image_ids(name):
  """Get the 1-based image indices for runs in a given experiment.

    Args:
      name (str) : Name of experiment ("rest" or name of task) to load
    Returns:
      run_ids (list of int) : Numeric ID for experiment image files

  """
  run_ids = [
    i for i, code in enumerate(BOLD_NAMES, 1) if name.upper() in code
  ]
  if not run_ids:
    raise ValueError(f"Found no data for '{name}''")
  return run_ids



def load_timeseries(subject, name, runs=None, concat=True, remove_mean=True):
  """Load timeseries data for a single subject.
  
  Args:
    subject (int): 0-based subject ID to load
    name (str) : Name of experiment ("rest" or name of task) to load
    run (None or int or list of ints): 0-based run(s) of the task to load,
      or None to load all runs.
    concat (bool) : If True, concatenate multiple runs in time
    remove_mean (bool) : If True, subtract the parcel-wise mean

  Returns
    ts (n_parcel x n_tp array): Array of BOLD data values

  """
  # Get the list relative 0-based index of runs to use
  if runs is None:
    runs = range(N_RUNS_REST) if name == "rest" else range(N_RUNS_TASK)
  elif isinstance(runs, int):
    runs = [runs]

  # Get the first (1-based) run id for this experiment 
  offset = get_image_ids(name)[0]

  # Load each run's data
  bold_data = [
      load_single_timeseries(subject, offset + run, remove_mean) for run in runs
  ]

  # Optionally concatenate in time
  if concat:
    bold_data = np.concatenate(bold_data, axis=-1)

  return bold_data


def load_single_timeseries(subject, bold_run, remove_mean=True):
  """Load timeseries data for a single subject and single run.
  
  Args:
    subject (int): 0-based subject ID to load
    bold_run (int): 1-based run index, across all tasks
    remove_mean (bool): If True, subtract the parcel-wise mean

  Returns
    ts (n_parcel x n_timepoint array): Array of BOLD data values

  """
  bold_path = f"{HCP_DIR}/subjects/{subject}/timeseries"
  bold_file = f"bold{bold_run}_Atlas_MSMAll_Glasser360Cortical.npy"
  ts = np.load(f"{bold_path}/{bold_file}")
  if remove_mean:
    ts -= ts.mean(axis=1, keepdims=True)
  return ts

In [3]:
import pandas as pd
import numpy as np
import os

# The download cells will store the data in nested directories starting here:
HCP_DIR = "./hcp"
if not os.path.isdir(HCP_DIR):
  os.mkdir(HCP_DIR)

# The data shared for NMA projects is a subset of the full HCP dataset
N_SUBJECTS = 339

# The data have already been aggregated into ROIs from the Glasesr parcellation
N_PARCELS = 360

# The acquisition parameters for all tasks were identical
TR = 0.72  # Time resolution, in sec

# The parcels are matched across hemispheres with the same order
HEMIS = ["Right", "Left"]

# Each experiment was repeated multiple times in each subject
N_RUNS_REST = 4
N_RUNS_TASK = 2


BOLD_NAMES = [
  "rfMRI_REST1_LR", "rfMRI_REST1_RL",
  "rfMRI_REST2_LR", "rfMRI_REST2_RL",
  "tfMRI_MOTOR_RL", "tfMRI_MOTOR_LR",
  "tfMRI_WM_RL", "tfMRI_WM_LR",
  "tfMRI_EMOTION_RL", "tfMRI_EMOTION_LR",
  "tfMRI_GAMBLING_RL", "tfMRI_GAMBLING_LR",
  "tfMRI_LANGUAGE_RL", "tfMRI_LANGUAGE_LR",
  "tfMRI_RELATIONAL_RL", "tfMRI_RELATIONAL_LR",
  "tfMRI_SOCIAL_RL", "tfMRI_SOCIAL_LR"
]



subjects = range(N_SUBJECTS)

timeseries_rest = []
for subject in subjects:
  ts_concat = load_timeseries(subject, "rest")
  timeseries_rest.append(ts_concat)

timeseries_rest_array = np.array(timeseries_rest)

all_fc_data = np.zeros((339, 64980))

fc = np.zeros((N_SUBJECTS, N_PARCELS, N_PARCELS))
for sub, ts in enumerate(timeseries_rest):
  fc[sub] = np.corrcoef(ts)

fc.shape

(339, 360, 360)

# Let's predict the behavioural performance (e.g. fluid intelligence)

In [62]:
import tensorflow as tf 


all_behav_data = pd.read_csv('filtered_behavioural.csv', sep=';')
behav = 'PMAT24_A_CR'
all_behav_data[behav].fillna((all_behav_data[behav].mean()), inplace=True)


X = fc
y = np.array(all_behav_data[behav])


X = X.reshape(-1,360, 360, 1)
                

input_shape = (360, 360)

model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    optimizer='sgd',
    loss='mse',
    metrics=["mean_squared_error"])

model.fit(X, y, batch_size=32, epochs=3, validation_split=0.3)



Train on 237 samples, validate on 102 samples
Epoch 1/3
237/237 [==============================] - 199s 841ms/sample - loss: 279.9528 - mean_squared_error: 279.9528 - val_loss: 281.4314 - val_mean_squared_error: 281.4314
Epoch 2/3
237/237 [==============================] - 181s 763ms/sample - loss: 277.7547 - mean_squared_error: 277.7546 - val_loss: 281.4314 - val_mean_squared_error: 281.4314
Epoch 3/3
237/237 [==============================] - 190s 803ms/sample - loss: 277.7547 - mean_squared_error: 277.7546 - val_loss: 281.4314 - val_mean_squared_error: 281.4314


# Let's try to predict gender instead     


In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical


all_behav_data = pd.read_csv('filtered_behavioural.csv', sep=';')
all_behav_data['Gender'].replace(['F','M'],[0,1],inplace=True)

behav = 'Gender'
all_behav_data[behav].fillna((all_behav_data[behav].mean()), inplace=True)
X = fc
y = np.array(all_behav_data[behav])

y = to_categorical(y)


X = X.reshape(-1,360, 360, 1)
                

input_shape = (360, 360)

model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(2))
model.add(Activation('sigmoid'))
#model.compile(loss='binary_crossentropy',
 #             optimizer='adam',
  #            metrics=['accuracy'])

opt = SGD(lr=0.001)
model.compile(loss = "binary_crossentropy", optimizer = opt, metrics=['accuracy'])

model.fit(X, y, batch_size=32, epochs=20, validation_split=0.3)

Train on 237 samples, validate on 102 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
237/237 [==============================] - 224s 946ms/sample - loss: 0.6945 - acc: 0.4852 - val_loss: 0.7016 - val_acc: 0.4314
Epoch 2/20
237/237 [==============================] - 193s 813ms/sample - loss: 0.6938 - acc: 0.5148 - val_loss: 0.6902 - val_acc: 0.5637
Epoch 3/20
237/237 [==============================] - 186s 784ms/sample - loss: 0.6894 - acc: 0.5612 - val_loss: 0.6815 - val_acc: 0.5686
Epoch 4/20
237/237 [==============================] - 188s 792ms/sample - loss: 0.6854 - acc: 0.5422 - val_loss: 0.6819 - val_acc: 0.5588
Epoch 5/20
237/237 [==============================] - 187s 789ms/sample - loss: 0.6826 - acc: 0.5506 - val_loss: 0.6776 - val_acc: 0.5686
Epoch 6/20
237/237 [==============================] - 209s 880ms/sample - loss: 0.6826 - acc: 0.5886 - val_loss: 0.6773 - val_acc: 0.5686
Epoch 7/20
237/237 [==============================] - 179s 754ms/sample - loss